In [2]:
import torch
import torch.nn as nn
from models.dpt import DPTSegmentationWithCarbon
from dataset import CarbonDataset
from torch.utils.data import DataLoader
from torchvision import transforms

kwargs = {
    "num_classes": 7,
}

model = DPTSegmentationWithCarbon(num_classes=kwargs["num_classes"])
path = "Dataset/Training/image/SN10_Forest_IMAGE"
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
data_loader = DataLoader(CarbonDataset(path,transform, mode = "Train"), batch_size=2, shuffle=True)
batch = data_loader.__iter__().__next__()
loss = model.validation_step(batch)


SEG num_classes 7
DPT num_classes 7
BASE num_classes 7
Dataset/Training/image/SN10_Forest_IMAGE Done.
Dataset/Training/image/SN10_Forest_SH Done.
Dataset/Training/label/SN10_Forest_Carbon Done.
Dataset/Training/label/SN10_Forest_GT Done.


RuntimeError: output with shape [1, 256, 256] doesn't match the broadcast shape [3, 256, 256]